# Práctica 2: Algoritmos metaheurísticos

## Sistemas Inteligentes

### Curso académico 2022-2023

#### Profesorado:

* Juan Carlos Alfaro Jiménez (`JuanCarlos.Alfaro@uclm.es`)
* Guillermo Tomás Fernández Martín (`Guillermo.Fernandez@uclm.es`)
* María Julia Flores Gallego (`Julia.Flores@uclm.es`)
* Ismael García Varea (`Ismael.Garcia@uclm.es`)
* Luis González Naharro (`Luis.GNaharro@uclm.es`)
* Aurora Macías Ojeda (`Profesor.AMacias@uclm.es`)
* Marina Sokolova Sokolova (`Marina.Sokolova@uclm.es`)

## 0. Preliminares

Antes de comenzar con el desarrollo de esta práctica es necesario **descargar**, en **formato `.py`**, el código de la **práctica anterior** con el nombre **`utils.py`**. Para ello, pulsamos, en la libreta de la primera práctica, **`File > Export .py`**.

Una vez hemos descargado y nombrado correctamente el fichero, lo **añadimos** al espacio de trabajo de la **libreta** de la práctica **actual** a través de **`Attached data > Notebook files > Upload files`** y subimos el fichero `utils.py` descargado en el paso anterior.

Tras esto, debemos **cambiar** el **constructor** de la clase **`Problem`** para que **reciba** directamente el **problema** a resolver, **en lugar de cargarlo desde** un **fichero**. Esto se debe a que los algoritmos metaheurísticos van a tener que resolver, en múltiples ocasiones, este problema. De esta manera nos **evitamos** la **carga computacional extra** que implica **leer** el problema desde un **fichero**. Además, también es necesario **comentar** cualquier línea de **código** que **imprima estadísticas** para evitar salidas largas.

## 1. Introducción

En esta práctica, vamos a **resolver** un **problema** de **optimización combinatoria** mediante **algoritmos metaheurísticos**. En particular, vamos a implementar **algoritmos genéticos** para abordar el **problema** de **ruteo** de **vehículos**. En este, **varios vehículos** con **capacidad limitada** deben **recoger paquetes** en **diferentes ubicaciones** y **trasladarlos** a una **sede central** de recogida.

Además, se **analizará** y **comparará** el **rendimiento** de **diferentes algoritmos genéticos** (mediante la modificación de los pasos correspondientes) en diferentes instancias del problema.

---

## 2. Descripción del problema

El concepto de mapa que vamos a utilizar es similar al de la primera práctica. Este se representa mediante un **grafo**, donde los **nodos** representan **ciudades** y los **enlaces** indican la existencia de una **carretera en ambos sentidos** entre dos ciudades. Además, los **enlaces** tienen un **peso** asociado indicando la **distancia real** entre las dos ciudades. Al mismo tiempo, se proporciona una **sede central**, que se trata de la ciudad donde se deben dejar los paquetes. A su vez, se dispone de una **flota de vehículos** con **capacidad limitada** que deben **recoger** los **paquetes** en las ciudades correspondientes y que **inicialmente** están aparcados en la **sede central**. **En caso de que a la hora de recoger un paquete se supere la capacidad del vehículo correspondiente, este debe volver a la sede central a descargar todos los paquetes, considerando el coste que implicaría volver**.

Un mapa es un problema en particular, pero diferentes paquetes, capacidades de vehículos y ubicación de la sede central pueden dar lugar a diferentes instancias del problema. Por tanto, el **objetivo** en este problema es **recoger todos los paquetes de tal manera que los vehículos recorran la menor distancia posible**.

**Con el objetivo de simplificar la práctica en evaluación continua, se asume que se cuenta con un único vehículo. No obstante, esto podría cambiar para la evaluación no continua.**

---

## 3. Desarrollo de la práctica

Durante el desarrollo de la práctica se va a proporcionar un conjunto de mapas, sobre los cuáles se debe resolver el problema de optimización combinatoria correspondiente. Es importante destacar que la **dimensionalidad** del **problema** (número de ciudades, carreteras y paquetes) puede ser **variable**, por lo que los diferentes **algoritmos genéticos** deben ser lo suficientemente **eficientes** para que puedan **resolver** los **problemas** en un **tiempo razonable**.

**Además, algunos escenarios se van a guardar para las entrevistas de prácticas, por lo que el código debe ser lo más general posible para cargarlos de manera rápida y sencilla.**

### 3.1. Entrada

Cada escenario tendrá un fichero `.json` asociado con la siguiente estructura:

```JSON
{
    "map": {
        "cities": [
            {
                "id": id_city_0,
                "name": name_city_0,
                "lat": latitude_city_0,
                "lon": longitude_city_0
            }
        ],
        "roads": [
            {
                "origin": origin_city_id,
                "destination": destination_city_id,
                "distance": road_distance
            }
        ]
    },
    "warehouse": warehouse_city_id,
    "vehicles": [
        {
            "id": id_vehicle_0,
            "capacity": capacity_vehicle_0
        }
    ]
    "parcels": [
        {
            "id": id_parcel_0,
            "city": parcel_city_id,
            "weight": weight_parcel_0
        }
    ]
}
```

Hay cuatro elementos principales en el fichero:

* `map`: Un diccionario con el mapa, cuya descripción es la misma que la de la primera práctica
* `warehouse`: Identificador de la ciudad donde se encuentra la sede central
* `vehicles`: Lista de vehículos disponibles
* `parcels`: Lista de paquetes a recoger

Por su parte, `vehicles` contiene:

* `id`: Identificador del vehículo
* `capacity`: Capacidad máxima del vehículo

Y `parcels`:

* `id`: Identificador del paquete
* `city`: Ciudad donde se encuentra el paquete
* `weight`: Peso del paquete

**Para añadir los ficheros con los problemas al espacio de trabajo se debe usar el mismo procedimiento anterior.**

---

## 4. Plan de trabajo

### 4.1. Formalización del problema

Para resolver cualquier problema de optimización combinatoria en primer lugar hay que definir como vamos a **codificar** las **soluciones** al problema. Si bien es algo que se deja a criterio propio, se plantea la siguiente pregunta, **¿cuál puede ser la mejor representación para una secuencia de paquetes a recoger?**

Se puede comprobar si la respuesta es correcta introduciéndola en la variable `answer` del siguiente fragmento de código:

In [44]:
# Third party
import hashlib
import json 
import random
import numpy as np
import time
import concurrent.futures
from multiprocessing import Pool
import threading

In [45]:
check_answer = lambda answer, hashed: "The answer is " + ("" if hashlib.md5(answer).hexdigest() == hashed else "in") + "correct."

In [46]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

hashed = "90d377b31e1ac26d0d10d5612ce33ccc"  # The hashed answer
print(hashed)

check_answer(answer, hashed)

90d377b31e1ac26d0d10d5612ce33ccc


'The answer is incorrect.'

### 4.2. Implementación

A continuación se proporciona la estructura de clases recomendada para resolver el problema en cuestión. Tendréis que completar las siguientes clases de acuerdo con los algoritmos estudiados en teoría. **Debéis incluir en la siguiente celda todas las librerías que vayáis a utilizar para mantener la libreta lo más organizada posible**:

In [47]:
# Importamos la clase utils.py en la carpeta actual
import utils as ut

#### Clase `Individual`

Esta clase proporciona la **codificación** de un **individuo** de la **población**.

Los **métodos obligatorios** que se deben añadir son:

* ``__init__(self, num_genes, generation_type, crossover_type, mutation_type)``: Inicializa el **número** de **genes** del **individuo** y el **tipo** de **operación** de **generación**, **cruce** y **mutación**. Ademas, genera la **solución** que **representa** el **individuo**.
* ``generate(num_genes, generation_type)``: Método estático para **generar** una **solución** del tamaño proporcionado de acuerdo con el tipo de operación de generación.
* ``crossover(self, individual)``: **Cruza** el **individuo actual** con el **individuo** de **entrada** de acuerdo con el tipo de operación de cruce.
* ``mutation(self)``: **Muta** el **individuo** de acuerdo con el tipo de operación de mutación.
* ``evaluate(self, problem)``: **Evalua** el **individuo** usando el **problema** a **resolver**.

Y los **métodos recomendados** son:

* ``__str__(self)``: **Representación** en formato de **cadena** de **caracteres** de un **individuo**. Método útil para depurar una lista de individuos.
* ``__repr__(self)``: **Método** invocado cuando se ejecuta **``print``** sobre el **individuo**. Método útil para depurar un solo individuo.

In [48]:
class Individual:

    # =============================================================================
    # Constructor
    # =============================================================================

    # Creamos un objeto de random para poder utilizarlo en los metodos estaticos
    #random = random.Random(123)

    def __init__(self, num_genes, generation_type="random", crossover_type = None, mutation_type = None):
        self.num_genes = num_genes
        self.generation_type = generation_type
        self.crossover_type = crossover_type
        self.mutation_type = mutation_type

        self.fitness = 0
        
        self.genes = []


    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # Método estático para generar una solución del tamaño proporcionado de acuerdo con el tipo de operación 
    # de generación.
    # A este metodo le pasamos num_genes, generation_type y un objeto de la clase random
    # @staticmethod
    def generate(num_genes, generation_type="random"):
        # Hay diferentes tipos de generación de individuos -> aleatoria, por distribucion, por enfoque, por seleccion...
        
        # Generación aleatoria -> Se generan los genes aleatoriamente

        # Generacion aleatoria para numeros enteros        
        if generation_type == "random":
            # Creamos un individuo vacio
            individual = Individual(num_genes, generation_type)

            # Generamos los genes aleatoriamente
            # Para ello utilizaremos la funcion random de la libreria random
            # for i in range(num_genes):
            #     individual.genes.append(random.random())

            # Generamos los genes aleatoriamente de tipo entero entre el rango de 0 a num_genes
            # for i in range(num_genes):
                # individual.genes.append(random.randint(0, num_genes - 1))  

            # Generamos los genes de forma aleatoria sin que se repitan
            individual.genes = random.sample(range(num_genes), num_genes)

            return individual
        

    # Método para realizar el cruce de dos individuos de acuerdo con el tipo de operación de cruce.
    def crossover(self, individual):
        # Tenemos diferentes tipos de operadores de cruce nosotros utilizaremos el que se utiliza para situaciones 
        # reales, el cruce 2PCX (Two Point Center Crossover)

        # Creamos los puntos de corte aleatoriamente, para hacer que siempre se generen dos puntos de corte debemos evitar
        # que los puntos de corte sean iguales y que los puntos de corte sean el primer o ultimo gen y que los puntos de corte
        # es decir que no sean 0 o num_genes - 1 

        cut_points = random.sample(range(1, self.num_genes - 1), 2)

        # cut_points = random.sample(range(1, self.num_genes - 1), 2)

        # Asignamos los puntos de corte segun el orden de menor a mayor
        cut_point_1, cut_point_2 = min(cut_points), max(cut_points)

        # print(cut_point_1, cut_point_2)
        # cut_point_1 = 1
        # cut_point_2 = 4

        list_1 = self.genes[cut_point_1:cut_point_2]
        list_2 = individual.genes[cut_point_1:cut_point_2]

        # # list_center_1 = [x for x in individual.genes if x in list_1]
        # list_center_1 = []
        # for num in individual.genes:
        #     if num in list_1:
        #         list_center_1.append(num)

        # list_center_2 = []
        # for num in self.genes:
        #     if num in list_2:
        #         list_center_2.append(num)

        # list_center_2 = [x for x in self.genes if x in list_2]

        child_1 = self.genes[:cut_point_1] + [num for num in individual.genes if num in list_1] + self.genes[cut_point_2:]
        individual.genes = individual.genes[:cut_point_1] + [num for num in self.genes if num in list_2] + individual.genes[cut_point_2:]
        self.genes = child_1

        return self, individual


    # def crossover(self, individual):
    #     # Tenemos diferentes tipos de operadores de cruce nosotros utilizaremos el que se utiliza para situaciones 
    #     # reales, el cruce 2PCX (Two Point Center Crossover)

    #     # Creamos los puntos de corte aleatoriamente, para hacer que siempre se generen dos puntos de corte debemos evitar
    #     # que los puntos de corte sean iguales y que los puntos de corte sean el primer o ultimo gen y que los puntos de corte
    #     # es decir que no sean 0 o num_genes - 1 

    #     cut_points = random.sample(range(1, self.num_genes - 1), 2)

    #     # Asignamos los puntos de corte segun el orden de menor a mayor
    #     cut_point_1, cut_point_2 = min(cut_points), max(cut_points)

    #     # Creamos los hijos
    #     child_1 = self.genes[:cut_point_1] + [x for x in individual.genes if x in self.genes[cut_point_1:cut_point_2]] + self.genes[cut_point_2:]
    #     individual.genes = individual.genes[:cut_point_1] + [x for x in self.genes if x in individual.genes[cut_point_1:cut_point_2]] + individual.genes[cut_point_2:]
    #     self.genes = child_1
        
    #     return self, individual


    # Método para realizar la mutación de un individuo de acuerdo con el tipo de operación de mutación.
    def mutation(self):
        
        # Tenemos diferentes mecanismos de mutación para este problema utilizaremos permutaciones, en concreto
        # Intercambio e inserción
        # 1. Intercambio -> Seleccionamos dos genes aleatoriamente y los intercambiamos
        # 2. Inserción -> Seleccionamos un gen aleatoriamente y lo insertamos en una posición aleatoria
        

        # Intercambio de genes utilizando libreria random con shuffle y choice
        # Intercambio de genes

        # Seleccionamos el tipo de mutación
        mutation_type = random.randint(0, 1)

        mutation_type = 0

        # Intercambio
        if mutation_type == 0:
            # print("\nINTERCAMBIO\n")
            # Seleccionamos dos genes aleatoriamente
            genes = random.sample(range(self.num_genes), 2)
            # Intercambiamos los genes
            self.genes[genes[0]], self.genes[genes[1]] = self.genes[genes[1]], self.genes[genes[0]]
        
        # Inserción
        else:
            # print("\nINSERCION\n")
            # Seleccionamos un gen aleatoriamente
            gene = random.randint(0, self.num_genes - 1)
            # print("GEN: ", gene)
            # Seleccionamos una posición aleatoria
            position = random.randint(0, self.num_genes - 1)
            # print("POSICION: ", position)
            # Insertamos el gen en la posición
            self.genes.insert(position, self.genes.pop(gene))

        return self.genes

        

        
        
    
    def evaluate(self, problem):
        
        # Evaluamos el individuo
        self.fitness = problem.evaluate(self.genes)

        return self.fitness
        
        
    def evaluate_first_number(self):
        # Evaluamos el individuo en base al primer numero de la lista de genes
        self.fitness = self.genes[0]

        return self.fitness
    

    # =============================================================================
    # Recommended methods
    # =============================================================================
    

    def __str__(self):
        return f' Individual: {self.genes}'

    def __repr__(self):
        return f' Individual: {self.genes}'

    

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [49]:
# TODO: Test here the methods to generate a solution

# Creamos un individuo
individual = Individual.generate(10, "random")

# Imprimimos el individuo
print(individual)




 Individual: [0, 8, 3, 6, 7, 5, 1, 4, 9, 2]


In [50]:
# TODO: Test here the methods to cross individuals

# Creamos dos individuos
individual_1 = Individual.generate(5, "random")
individual_2 = Individual.generate(5, "random")

# Imprimimos los individuos
print("Individuo 1\n", individual_1)
print("Individuo 2\n", individual_2)

# Realizamos el cruce
child_1, child_2 = individual_1.crossover(individual_2)

# Imprimimos los hijos
print("Hijo 1", child_1)
print("Hijo 2", child_2)


Individuo 1
  Individual: [3, 2, 0, 1, 4]
Individuo 2
  Individual: [1, 2, 0, 4, 3]
Hijo 1  Individual: [3, 2, 0, 1, 4]
Hijo 2  Individual: [1, 2, 0, 4, 3]


In [51]:
# TODO: Test here the methods to mutate an individual

# Creamos un individuo
individual = Individual.generate(5, "random")

# Imprimimos el individuo
print("Individuo\n", individual)

# Realizamos la mutación
individual.mutation()

# Imprimimos el individuo mutado
print("Individuo mutado\n", individual)



Individuo
  Individual: [2, 4, 3, 1, 0]
Individuo mutado
  Individual: [2, 1, 3, 4, 0]


#### Clase `Genetic`

Esta clase implementa un **esquema** básico de **algoritmo genético**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)``: Inicializa el **tamaño** de la **población**, el **tipo** de **operación** de **selección**, **cruce**, y **mutación**, así como la **probabilidad** de aplicar las operaciones de **cruce** y **mutación**. Además, también inicializa el **número** de **mejores soluciones** de la **población actual** que se **mantienen** en la **siguiente población** y una **semilla** para garantizar que los **experimentos** son **reproducibles**. **Nótese que puede ser necesario añadir más argumentos si así se requiere**.
* ``def __call__(self, problem)``: **Método** que se **ejecuta** cuando se llama a un **objeto** de la **clase como** si fuese una **función**. En este **método** se debe **implementar** el **esquema básico** de un **algoritmo genético** que se encargue de ejecutar los pasos correspondientes. 
* ``def generate_population(self, problem)``: **Genera** la **población inicial** de acuerdo con el **problema** a resolver.
* ``def select_population(self, population, scores)``: **Selecciona** los **padres** a utilizar para la operación de cruce.
* ``def crossover(self, population)``: **Cruza pares** de **padres** teniendo en cuenta la probabilidad de cruce.
* ``def mutation(self, population)``: **Muta** los **individuos cruzados** teniendo en cuenta la probabilidad de mutación.
* ``def evaluate(self, population, problem)``: **Evalua** los **nuevos individuos** de acuerdo con el problema a resolver.
* ``def combine(self, population)``: **Forma** la **nueva generación** de acuerdo con el número de mejores individuos de la población actual a mantener en la siguiente.

In [52]:
class Genetic:

    # =============================================================================
    # Constructor
    # =============================================================================

    def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state, parrallel=True):
        
        # Tamaño de la población
        self.population_size = population_size

        # Numero de generaciones
        self.num_generations = num_generations

        # Tipo de selección
        self.selection_type = selection_type

        # Tipo de cruce
        self.crossover_type = crossover_type

        # Probabilidad de cruce
        self.crossover_probability = crossover_probability

        # Tipo de mutación
        self.mutation_type = mutation_type

        # Probabilidad de mutación
        self.mutation_probability = mutation_probability

        # Lista de individuos
        self.population = []

        # Si queremos mantener el elitismo, es decir, el mejor individuo de la población anterior 
        self.keep_elitism = keep_elitism
        
        # Semilla aleatoria, la semilla aleatoria es un número que se utiliza para inicializar un generador de números pseudoaleatorios.
        # Esto nos ayuda a obtener los mismos resultados cada vez que ejecutamos el algoritmo
        random.seed(random_state)

        # Si queremos que el algoritmo se ejecute en paralelo
        self.parrallel = parrallel


    def __call__(self, problem):
        # self.problem = problem
        # Call es el encargado de llamar a los métodos de la clase Genetic y de esta forma poder ejecutar el algoritmo genético
        # Para ello seguiremos los siguientes pasos:

        # 1. Inicializar la población: utilizar el método generate_population para generar una población de individuos de tamaño 
        # population_size, utilizando el problema especificado en el argumento problem.

        # 2. Evaluar la población: utilizar el método evaluate para evaluar a cada individuo de la población y obtener sus puntuaciones.

        # 3. Repetir el siguiente proceso durante un número determinado de generaciones (num_generations):

        # 4. Seleccionar individuos de la población: utilizar el método select_population para seleccionar un conjunto de individuos de la 
        # población utilizando el método de selección especificado en el argumento selection_type.

        # 5. Aplicar crossover: utilizar el método crossover para aplicar el tipo de crossover especificado en el argumento crossover_type a 
        # los individuos seleccionados, con una probabilidad especificada en el argumento crossover_probability.

        # 6. Aplicar mutación: utilizar el método mutation para aplicar el tipo de mutación especificado en el argumento mutation_type a los 
        # individuos resultantes del crossover, con una probabilidad especificada en el argumento mutation_probability.

        # 7. Evaluar la nueva población: utilizar el método evaluate para evaluar a cada individuo de la nueva población y obtener sus puntuaciones.

        # 8. Combinar la población: utilizar el método combine para combinar la población actual con la nueva población generada, 
        # teniendo en cuenta el argumento keep_elitism para determinar si se deben conservar los individuos más aptos de la población actual.
                

        # 1. Inicializar la población
        self.generate_population(problem)

        # 2. Evaluar la población, al llamar a la función evaluate se le asigna a la variable fitness en la clase Individual la puntuación
        if self.parrallel:
            # Si queremos que el algoritmo se ejecute en paralelo
            self.evaluate_parallel(self.population, problem)
        else:
            self.evaluate(self.population, problem)

        # 3. Repetir el siguiente proceso durante un número determinado de generaciones (num_generations)
        for _ in range(self.num_generations):
                
                # 4. Seleccionar individuos de la población
                selected_population = self.select_population(self.population)
    
                # 5. Aplicar crossover
                crossed_population = self.crossover(selected_population)
    
                # 6. Aplicar mutación
                mutated_population = self.mutation(crossed_population)
    
                # 7. Evaluar la nueva población
                if self.parrallel:
                    # Si queremos que el algoritmo se ejecute en paralelo
                    self.evaluate_parallel(mutated_population, problem)
                else:
                    self.evaluate(mutated_population, problem)
    
                # 8. Combinar la población
                self.population = self.combine4(mutated_population)

        # Devolvemos la mejor solución, es decir la que tenga la puntuación mas baja
        return min(self.population, key=lambda x: x.fitness)
        # return self.population





    # =============================================================================
    # Mandatory methods
    # =============================================================================

    def generate_population(self, problem):
        
        # Creamos la problacion inicial
        # population = [Individual.generate(problem.num_genes, "random") for _ in range(self.population_size)]

        # Esta forma crea una población de individuos aleatorios
        # self.population = [Individual.generate(problem, "random") for _ in range(self.population_size)]

        # Ahora creamos la poblacion para el problema en si, es decir, CVRP
        # Si problem es de tipo int es que es un problema de prueba 
        if isinstance(problem, int):
            self.population = [Individual.generate(problem, "random") for _ in range(self.population_size)]
        else:
            self.population = [Individual.generate(len(problem.parcels_dict), "random") for _ in range(self.population_size)]


        # return self.population
        
    @staticmethod
    def select_tournament(population):
        # Seleccion por torneo 
        # Seleccionamos dos individuos aleatorios de la población y nos quedamos con el mejor 

        # Seleccionamos dos individuos aleatorios de la población
        individuals = random.sample(population, 2)

        # Nos quedamos con el mejor
        best_individual = min(individuals, key=lambda x: x.fitness)

        return best_individual


    def select_population(self, population, num_individuos_selected=None):
        # Selecciona un conjunto de individuos de la población utilizando el método de selección especificado en el argumento selection_type.
        selected_population = set()

        # El numero de individuos seleccionados debe ser menor o igual que el tamaño de la población, mayor que 0 y par
        if num_individuos_selected is None:
            # Si no se especifica el numero de individuos seleccionados, se selecciona la mitad de la población
            num_individuos_selected = self.population_size // 2
            
    
        # Una vez especificado el numero de individuos seleccionados, comprobamos que sea menor o igual que el tamaño de la población, mayor que 0 y par
        if num_individuos_selected > self.population_size or num_individuos_selected <= 0:
            raise ValueError("El numero de individuos seleccionados debe ser menor o igual que el tamaño de la población, mayor que 0 y par")
        if num_individuos_selected % 2 != 0:
            # Si el numero de individuos seleccionados no es par, lo hacemos par
            # Para ello podemos restarle 1 o sumarle 1 sumaremos siempre 1 a menos que num_individuos_selected sea igual que self.population_size
            if num_individuos_selected == self.population_size or num_individuos_selected == self.population_size - 1:
                num_individuos_selected -= 1
            else:
                num_individuos_selected += 1



        if self.selection_type == "torneo":
            # Seleccion por torneo
            for _ in range(num_individuos_selected):
                # Añadimos el individuo seleccionado por torneo a la población
                selected_population.add(self.select_tournament(population))


        elif self.selection_type == "roulette":
            # Seleccion por ruleta
            pass


        elif self.selection_type == "ranking":
            # Seleccion por ranking
            pass


        elif self.selection_type == "random":
            # Seleccion aleatoria
            pass
        
        else:
            raise Exception("Invalid selection type")

        # Convertimos la población seleccionada en una lista
        selected_population = list(selected_population)

        return selected_population


    def crossover(self, selected_population):
        # Cruza pares de padres teniendo en cuenta la probabilidad de cruce.
        # Para la seleccion de los padres utilizamos el metodo select_population
        # Para el cruce utilizamos el metodo crossover de la clase Individual

        # El cruce se realiza teniendo en cuenta la probabilidad de cruce self.crossover_probability
        # Si la probabilidad de cruce es mayor que el numero aleatorio generado entonces se realiza el cruce
        # Si la probabilidad de cruce es menor que el numero aleatorio generado entonces no se realiza el cruce
        
        # Creamos un numero aleatorio entre 0 y 1 para realizar la probabilidad de cruce
        random_number = random.random()
        if random_number < self.crossover_probability:
            # Comprobamos la longitud de la población seleccionada
            # Si la longitud es par realizamos el cruce de todos los seleccionados en pares
            # Si la longitud es impar realizamos el cruce de todos los seleccionados en pares menos el ultimo
            # print("Cruce")

            if len(selected_population) % 2 == 0:
                # Creamos una lista con los padres en pares
                parents = [selected_population[i:i+2] for i in range(0, len(selected_population), 2)]
            else:
                # Creamos una lista con los padres en pares menos el ultimo
                parents = [selected_population[i:i+2] for i in range(0, len(selected_population) - 1, 2)]

            # Creamos una lista para los hijos
            childs = []

            for parent1, parent2 in parents:
                # Realizamos el cruce de los padres
                child1, child2 = parent1.crossover(parent2)

                # Añadimos los hijos a la lista de hijos
                childs += [child1, child2]
            
        else:
            # print("Cruce no realizado")
            # Si no se realiza el cruce, devolvemos la población seleccionada sin cruzar
            return selected_population
            

        return childs




    def mutation(self, population):
        # Añadimos una probabilidad de mutación
        for individual in population:
            if random.random() < self.mutation_probability:
                individual.mutation()

        return population


    # def evaluate(self, population, problem):
    #     # Evaluamos los nuevos individuos de acuerdo con el problema a resolver, devolviendo una lista con las puntuaciones de cada individuo
    #     scores = [problem.evaluate(individual) for individual in population]

    #     return scores

    def evaluate(self, population, problem):
        # Evaluamos los individuos de acuerdo con el problema a resolver
        for individual in population:
            if individual.fitness == 0:
                individual.evaluate(problem)



    # # Para mejorar la eficiencia del metodo evaluate, lo hacemos de forma paralela
    # def evaluate_parallel(self, population, problem):
    #     # Evaluamos los individuos de acuerdo con el problema a resolver
    #     with concurrent.futures.ProcessPoolExecutor() as executor:
    #         executor.map(lambda individual: individual.evaluate(problem), population)


    # POOL MULTIPROCCESING

    # def evaluate_individual(individual, problem):
    #     individual.evaluate(problem)

    # def evaluate_parallel(self, population, problem):
        
    #     # Creamos un pool de procesos con 4 procesos
    #     with Pool(4) as p:
    #         # Evaluamos los individuos de acuerdo con el problema a resolver
    #         for individual in population:
    #             if individual.fitness == 0:
    #                 p.apply_async(self.evaluate_individual, args=(individual, problem))

    #         # Esperamos a que todos los procesos terminen
    #         p.close()
    #         p.join()


    # THREADING
    def evaluate_parallel(self, population, problem):

        def evaluate_individual(individual, problem):
            individual.evaluate(problem)

        # Creamos una lista de hilos
        threads = []

        # Evaluamos los individuos de acuerdo con el problema a resolver
        for individual in population:
            if individual.fitness == 0:
                # Creamos un hilo para evaluar el individuo y lo añadimos a la lista de hilos
                thread = threading.Thread(target=evaluate_individual, args=(individual, problem))
                threads.append(thread)

        # Lanzamos todos los hilos
        for thread in threads:
            thread.start()

        # Esperamos a que todos los hilos terminen
        for thread in threads:
            thread.join()
   


    def combine(self, population, new_population):
        # Combina la población actual con la nueva población, devolviendo una nueva población con el tamaño especificado en el argumento population_size.
        
        # Cosas a tener en cuenta:
        # 1. La nueva población debe tener el tamaño especificado en el argumento population_size.
        # 2. Tenemos una variable self.keep_elitism que determina si debemos mantener los mejores individuos de la población actual en la nueva población.
            # 2.1 Si self.keep_elitism es True, debemos añadir los mejores individuos de la población actual a la nueva población,
            # es decir, los individuos con mayor fitness no deben ser sustituidos por los nuevos individuos.
            # 2.2 Si self.keep_elitism es False, no debemos añadir los mejores individuos de la población actual a la nueva población.
        # 3. La nueva población debe estar ordenada de mayor a menor fitness.
        
        # Tenemos una poblacion actual(population) y una nueva poblacion(new_population) que debemos combinar
        
        # Si self.keep_elitism es True, debemos añadir los mejores individuos de la población actual a la nueva población,

        if self.keep_elitism:
            # Ordenamos la poblacion actual de mayor a menor fitness
            population.sort(key=lambda x: x.fitness, reverse=True)
            # Añadimos los mejores individuos de la poblacion actual a la nueva poblacion
            new_population.extend(population[:self.population_size - len(new_population)])

        # Ordenamos la nueva poblacion de mayor a menor fitness
        new_population.sort(key=lambda x: x.fitness, reverse=True)
        # Devolvemos la nueva poblacion
        return new_population[:self.population_size]




    # Hacemos el metodo combine de nuevo teniendo en cuenta cuando self.keep_elitism es False
    def combine2(self, population, new_population):

        if self.keep_elitism:
            # Ordenamos la poblacion actual de mayor a menor fitness
            population.sort(key=lambda x: x.fitness, reverse=True)
            # Añadimos los mejores individuos de la poblacion actual a la nueva poblacion
            new_population.extend(population[:self.population_size - len(new_population)])
        else:
            # Ordenamos la nueva poblacion de mayor a menor fitness
            new_population.sort(key=lambda x: x.fitness, reverse=True)

            # Comprobamos si la nueva poblacion es mayor que la poblacion actual
            if len(new_population) > len(population):
                # Si es mayor, devolvemos la nueva poblacion
                return new_population[:self.population_size]
            else:
                # Si no es mayor, devolvemos una mezcla de la poblacion actual y la nueva poblacion seleccionandolos de forma aleatoria por torneo
                for _ in range(self.population_size - len(new_population)):
                    new_population.append(self.select_tournament(population))
                return new_population
            

        # Ordenamos la nueva poblacion de mayor a menor fitness 
        new_population.sort(key=lambda x: x.fitness, reverse=True)
        # Devolvemos la nueva poblacion
        return new_population[:self.population_size]



        # Hacemos el metodo combine de forma mas eficiente y sencilla, tenemos en cuenta tambien que no hace falta pasarle la poblacion actual
        # puesto que llamamos desde self y tenemos acceso a ella
    def combine3(self, new_population):
        # Si self.keep_elitism es True, debemos añadir los mejores individuos de la población actual a la nueva población,
        if self.keep_elitism:
            # Añadimos los mejores individuos de la poblacion actual a la nueva poblacion
            new_population.extend(self.population[:self.population_size - len(new_population)])

        # Ordenamos la nueva poblacion de mayor a menor fitness
        new_population.sort(key=lambda x: x.fitness, reverse=True)
        # Devolvemos la nueva poblacion
        return new_population[:self.population_size]

    
    # Hacemos el combine2 con los mismos parametros que el combine3
    def combine4(self, new_population):
        
        if self.keep_elitism:
            # Añadimos los mejores individuos de la poblacion actual a la nueva poblacion
            # Ordenamos la nueva poblacion de menor a mayor fitness
            new_population.sort(key=lambda x: x.fitness)
            self.population.sort(key=lambda x: x.fitness)
            population_size = self.population_size
            new_population_size = len(new_population)

            # Mezclamos el 50% de la poblacion actual con el 50% de la nueva poblacion (los mejores con los mejores de cada poblacion)
            new_population_2 = []

            population_size_50 = population_size // 2

            # Hacemos un conjunto de los elementos de la poblacion actual y la nueva poblacion
            population_set = set(self.population)

            num_added_new_p = 0

            # def add_individuals()

            # Si la nueva poblacion no tiene los elementos suficientes (50%) para completar la poblacion actual, añadimos los faltantes de la poblacion actual
            if new_population_size < population_size_50:
                
                # POBLACION NUEVA 
                # Añadimos el 50% de la nueva poblacion evitando que se repitan elementos
                for individual in new_population:
                    if individual not in population_set:
                        new_population_2.append(individual)
                        population_set.add(individual)
                        num_added_new_p += 1

            # Si tiene los elementos suficientes, añadimos mitad de la poblacion actual y mitad de la nueva poblacion
            else:
                # POBLACION NUEVA
                population_size_50_aux = population_size_50
                # Añadimos el 50% de la nueva poblacion evitando que se repitan elementos
                # new_population_2.extend(new_population[:population_size_50])
                for individual in new_population:
                    if individual not in population_set:
                        new_population_2.append(individual)
                        population_set.add(individual)
                        num_added_new_p += 1
                    if num_added_new_p == population_size_50:
                        break


            # POBLACION ACTUAL
            # Si la poblacion es impar sumamos 1
            if population_size % 2 != 0:
                # Añadimos el 50% de la poblacion actual + los que faltan de la nueva poblacion por estar repetidos en la poblacion actual
                new_population_2.extend(self.population[:population_size_50 + (population_size_50 - num_added_new_p) + 1])
            else:
                new_population_2.extend(self.population[:population_size_50 + (population_size_50 - num_added_new_p)])
                # Si no se han añadido todos los elementos de la nueva poblacion, añadimos los faltantes de la poblacion actual
                # if num_added_new_p <= population_size_50:
                #     # Añadimos a la nueva poblacion 2 los elementos faltantes de la poblacion actual
                #     new_population_2.extend(self.population[:population_size_50 - num_added_new_p])

            # Antes de añadirlo comprobamos elementos repetidos entre la poblacion actual y la nueva poblacion
            # Si hay elementos repetidos, no los añadimos a la nueva poblacion, si no se añade un elemento lo tenemos en cuenta en el for 
            # # para que la poblacion final tenga el tamaño deseado por tanto le añadimos 1 al tamaño de la poblacion
            # new_population_set = set(new_population)
            # for i in self.population[:self.population_size - len(new_population)]:
            #     if i not in new_population_set:
            #         new_population.append(i)
            #     else:
            #         self.population_size += 1
            
        else:
            # Comprobamos si la nueva poblacion es mayor que la poblacion actual
            if len(new_population) > len(self.population):
                # Si es mayor, devolvemos la nueva poblacion
                return new_population[:self.population_size]
            else:
                # Si no es mayor, devolvemos una mezcla de la poblacion actual y la nueva poblacion seleccionandolos de forma aleatoria por torneo
                # for _ in range(self.population_size - len(new_population)):
                #     new_population.append(self.select_tournament(self.population))
                # return new_population

                selected_individuals = [self.select_tournament(self.population) for _ in range(self.population_size - len(new_population))]
                return new_population + selected_individuals

            

        # Ordenamos la nueva poblacion de menor a mayor fitness
        new_population_2.sort(key=lambda x: x.fitness)
        # Devolvemos la nueva poblacion
        return new_population_2[:self.population_size]




**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [53]:
# TODO: Test here the methods to generate a population
# Probamos el metodo para generar una población inicial de Genetic 
# self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)
# A genetic le pasamos estos parametros
gn = Genetic(5, 10, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1, False)

# Creamos una población inicial
gn.generate_population(65)

# Imprimimos la población inicial
print('Población inicial')
for individual in gn.population:
    print(individual.genes)





Población inicial
[1, 4, 0, 5, 3, 2]
[3, 5, 4, 0, 2, 1]
[0, 3, 4, 2, 5, 1]
[2, 1, 0, 4, 3, 5]
[0, 4, 5, 1, 3, 2]


In [54]:
# TODO: Test here the methods to select individuals
# Probamos los metodos de seleccion de individuos 
# Para probar los metodos de seleccion de individuos, vamos a crear una poblacion de 10 individuos y vamos a seleccionar 5 individuos de la poblacion de 10 individuos

# Probamos el metodo para seleccionar los padres
parents = gn.select_population(gn.population)

# Imprimimos los padres seleccionados
print('Padres seleccionados')
for individual in parents:
    print(individual)

# print(type(parents))
# print(type(parents[0]))


Padres seleccionados
 Individual: [1, 4, 0, 5, 3, 2]
 Individual: [2, 1, 0, 4, 3, 5]


In [55]:
print(parents)

# Probramos el metodo para cruzar y mutar los padres
childs = gn.crossover(parents)

# print(childs)
# Imprimimos los hijos
print('Hijos cruzados')
for individual in childs:
    print(individual)
    # print(type(individual))

# Probamos el metodo para mutar los hijos
childs = gn.mutation(childs)

# Imprimimos los hijos mutados
print('Hijos mutados')
for individual in childs:
    print(individual)
    # print(type(individual))

    
# print(type(childs))
# print(type(childs[0]))
# print(type(gn.population))


[ Individual: [1, 4, 0, 5, 3, 2],  Individual: [2, 1, 0, 4, 3, 5]]
Hijos cruzados
 Individual: [1, 0, 4, 5, 3, 2]
 Individual: [2, 1, 0, 4, 3, 5]
Hijos mutados
 Individual: [1, 0, 4, 5, 3, 2]
 Individual: [2, 1, 0, 4, 3, 5]


In [56]:
# PRUEBAS INDIVIDUALES DE COMBINE DEBAJO DE CVRP

In [57]:
# Probamos el metodo __call__ de la clase Genetic encargado de ejecutar el algoritmo genetico completo
# gn = Genetic(10, 10, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1)

# Call crea la poblacion y hace los pasos necesarios para ejecutar el algoritmo genetico
# Le pasamos el problema que queremos resolver



#### Clase `CVRP`

Esta clase representa el problema en cuestión, esto es, el **problema** de **ruteo** de **vehículos**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, filename, algorithm)``: Inicializa el **problema** en cuestión y el **algoritmo** a usar para resolverlo. A su vez, se debe crear un **diccionario** que contenga como **clave** un **identificador** de **paquete** y como **valor** una **tupla** con la **ciudad** donde se encuentra dicho paquete y su **peso**.
* ``def __call__(self)``: **Resuelve** el **problema** en cuestión.
* ``def evaluate(self, solution)``: **Evalua** una **solución** para el **problema** en cuestión, **teniendo en cuenta** las **restricciones correspondientes**.
* ``def search(self, departure, goal)``: **Resuelve** un **problema** de **búsqueda** de **caminos** dada las **ciudades** de **salida** y **meta**.

**Nótese que se puede crear una estructura de datos para agilizar el proceso de búsqueda de caminos requerido por el algoritmo ¿cuál puede ser?**

In [58]:
# TODO: Introduce here the answer to use for the hashing
answer = "Hello"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

encoded = "0fea6a13c52b4d4725368f24b045ca84"  # The hashed answer
print(encoded)

check_answer(answer, encoded)

0fea6a13c52b4d4725368f24b045ca84


'The answer is incorrect.'

In [59]:
class CVRP:

    # =============================================================================
    # Constructor
    # =============================================================================
    
    # Sobreescribir el método __init__ de la clase problem ubicada en utils.py




# No necesitamos la primera funcion de carga de archivo, la ciudad objetivo ni ciudad inicial
    def __init__(self, filename="Spain/example.json", algorithm=None):

        self.filename = filename

        with open(filename, 'r', encoding='utf8') as file:
            problem = json.load(file)
        


        self.problema = ut.Problem(problem)

        self.genetic_algorithm = algorithm

        
        # Para el problema tenemos 4 elementos principales (JSON):
        # map -> mapa del problema (Práctica 1)
        # warehouses -> almacenes del problema (En este caso solo hay 1)
        # vehicles -> lista de vehículos disponibles
            # id -> identificador del vehículo
            # capacity -> capacidad del vehículo
        # parcels -> lista de paquetes a recoger
            # id -> identificador del paquete
            # city -> ciudad donde se encuentra el paquete 
            # weight -> peso del paquete
        
        # Warehouse -> Ciudad en la que se encuentra el almacén (en el problema solo habra 1 almacén) 
        # Elegimos el primer elemento de la lista de almacenes
        self.warehouse = problem['warehouse']

        # Vehicles -> Lista de vehículos disponibles (lista de diccionarios) (en los problemas solo habra 1 vehiculo)
        self.vehicles = problem['vehicles']

        # Parcels -> Lista de paquetes a recoger
        self.parcels = problem['parcels']

        # Creamos un diccionario para almacenar los paquetes, guardamos el id del paquete como clave y una tupla con la ciudad y el peso como valor
        self.parcels_dict = {parcel['id']:(parcel['city'], parcel['weight']) for parcel in self.parcels}


        # Numero de individuos evaluados reales
        self.real_evaluated_items = 0

        # Numero de individuos evaluados totales
        self.total_evaluated_items = 0

        # Diccionario para no explorar soluciones ya exploradas
        self.explored_solutions = {}

        # Diccionario de busquedas realizadas entre ciudades
        self.searches = {}

        # Cantidad de llamadas al metodo search reales 
        self.real_search_calls = 0

        # Cantidad de "llamadas" al metodo search totales
        self.total_search_calls = 0

        # Diccionario de estadisticas
        self.stats = {}

        
        # Matriz de costes entre ciudades
        
        


    # Resuelve el problema en cuestion
    def __call__(self):

        init_time = time.perf_counter()
        # Llamamos al algoritmo genetico para resolver el problema del CVRP y guardamos la solucion en la variable solution
        solution = self.genetic_algorithm(self)

        # population = self.genetic_algorithm(self)


        # Calculamos el tiempo que ha tardado en resolver el problema
        time_elapsed = time.perf_counter() - init_time


        # Guardamos las estadisticas en el diccionario de estadisticas
        self.stats['time'] = time_elapsed
        # Añadimos a la clave 'searches' el numero de llamadas al metodo search reales con clave 'real' y el numero de llamadas totales con clave 'total'
        self.stats['searches'] = {'real': self.real_search_calls, 'total': self.total_search_calls}
        # Añadimos a la clave 'population' el numero de individuos reales y el numero de individuos totales
        self.stats['population'] = {'real': self.real_evaluated_items, 'total': self.total_evaluated_items}

        # Ordenamos el diccionario de soluciones exploradas segun su fitness
        self.explored_solutions = dict(sorted(self.explored_solutions.items(), key=lambda item: item[1]))

        # Mostramos todos los caminos explorados 
        print("explore: ", self.explored_solutions)
        print("Explore len: ", len(self.explored_solutions))

        # Mostramos el que menor fitness tenga
        # print("explore_min: ", min(self.explored_solutions.values()))
        # Hacemos los print anteriores en uno solo
        print("\nProblem: ", self.filename, "\nSolution: ", solution, "\nStats: ", self.stats, "\nScore: ", solution.fitness)

        # Devolvemos el individuo con el menor fitness (explored_solutions es un diccionario ordenado por fitness) elemento 0 de la lista de claves
        return solution

        





    # Para problemas pequeños podemos hacer una matriz de costes entre ciudades
    def create_cost_matrix(self):
        pass






    # =============================================================================
    # Mandatory methods
    # =============================================================================


    # Funcion para resolver el problema del CVRP
    def solve(self, solution):
        # Resuelve el problema del CVRP
        # Recibe una solucion y devuelve una lista de rutas (lista de listas)
        pass
        

    def evaluate(self, solution):
        # Evalua una solucion para el problema del CVRP
        # El algoritmo genetico nos genera un individuo, una lista aleatoria con el orden de las ciudades a visitar

        # Cosas a tener en cuenta:
            # 1.1 Peso del paquete -> El peso del paquete debe ser menor que la capacidad del vehiculo, si no es asi, la solucion no es valida
            # 1.2 Peso del paquete -> Debemos comprobar antes de ir a la siguiente ciudad si el peso del paquete + el peso de los paquetes en el vehiculo 
            # supera la capacidad del vehiculo, si es asi, debemos volver al almacén para dejar los paquetes e ir a la siguiente ciudad
            # 2. Una solución sera mejor cuando la distancia total recorrida es la menor posible

        # Para resolver el problema del CVRP debemos recorrer la lista de ciudades que nos da el algoritmo genetico y comprobar si la solucion es valida
        # Si la solucion es valida, debemos calcular la distancia total recorrida y devolverla como fitness de la solucion
        
        # Inicializamos variables necesarias

        fitness = 0

        # Convertimos la solucion en una tupla para poder usarla como clave en el diccionario de soluciones exploradas
        tuple_solution = tuple(solution)

        # Comprobamos si el individuo ha sido evaluado previamente
        if tuple_solution in self.explored_solutions: 
            # Si ya ha sido evaluado, obtenemos el coste del diccionario y lo asignamos al fitness del individuo
            fitness = self.explored_solutions[tuple_solution]
            self.total_evaluated_items += 1

        else:
            # Si no ha sido evaluado, lo evaluamos
            # Creamos una variable para almacenar el peso total de los paquetes que llevamos en el vehiculo
            total_weight = 0

            # Creamos una lista para almacenar las rutas (orden a recoger del id de los paquetes)
            # routes = []

            # Creamos una lista para almacenar los paquetes que llevamos en el vehiculo
            # parcels = []

            # Creamos una variable para almacenar la ciudad en la se encuentra el vehiculo (warehouse)
            current_city = self.warehouse

            # Tenemos una lista de ciudades (solution) la cual debemos recorrer
            if solution is None:
                raise ValueError("Solution is None")
            else:

                
                # Recorremos la lista de paquetes
                for next_parcel in tuple_solution:
                    
                    # Siguiente ciudad
                    next_city = self.parcels_dict[next_parcel][0]

                    # Guardamos el peso del paquete de la siguiente ciudad + el peso total de los paquetes en el vehiculo
                    total_weight += self.parcels_dict[next_parcel][1]

                    # Comprobamos si el peso del paquete total supera la capacidad del vehiculo
                    if total_weight > self.vehicles[0]['capacity']:
                        # Si es asi, debemos volver al almacén para dejar los paquetes e ir a la siguiente ciudad

                        # Comprobamos si la ciudad del paquete actual y el almacen han sido calculadas previamente
                        if (current_city, self.warehouse) in self.searches:
                            # Si es asi, obtenemos el coste del diccionario y lo sumamos al fitness
                            fitness += self.searches[(current_city, self.warehouse)]
                            self.total_search_calls += 1
                        else: 
                            # Calculamos la distancia entre la ciudad en la que nos encontramos y la siguiente ciudad y la sumamos al fitness
                            fitness += self.search(current_city, self.warehouse)
                            # Añadimos el coste al diccionario de costes
                            self.searches[(current_city, self.warehouse)] = fitness
                            self.real_search_calls += 1

                        # Añadimos el id de los paquetes que llevamos en el vehiculo a la lista de rutas
                        # routes.append(parcels)
                        # Vaciamos la lista de paquetes que llevamos en el vehiculo
                        # parcels = []

                        # Actualizamos la ciudad en la que nos encontramos
                        current_city = self.warehouse
                        # Actualizamos el peso total de los paquetes que llevamos en el vehiculo
                        total_weight = 0

                    # Comprobamos si la ciudad actual y la siguiente ciudad han sido calculadas previamente
                    if (current_city, next_city) in self.searches:
                        # Si es asi, obtenemos el coste del diccionario y lo sumamos al fitness
                        fitness += self.searches[(current_city, next_city)]
                        self.total_search_calls += 1
                    else: 
                        # Calculamos la distancia entre la ciudad en la que nos encontramos y la siguiente ciudad y la sumamos al fitness
                        fitness += self.search(current_city, next_city)
                        # Añadimos el coste al diccionario de costes
                        self.searches[(current_city, next_city)] = fitness
                        self.real_search_calls += 1


                    # Viajamos a la siguiente ciudad
                    
                    # Añadimos el id del paquete de la siguiente ciudad a la lista de paquetes que llevamos en el vehiculo
                    # parcels.append(next_parcel)
                    # Actualizamos la ciudad actual
                    current_city = self.parcels_dict[next_parcel][0]

                # Una vez recorridas todas las ciudades, debemos volver al almacén para dejar los paquetes
                # Calculamos la distancia entre la ciudad en la que nos encontramos y el almacén, la sumamos al fitness
                # fitness += self.search(current_parcel, self.warehouse)
                # Añadimos el id de los paquetes que llevamos en el vehiculo a la lista de rutas
                # routes.append(parcels)

            # Añadimos el fitness al diccionario de soluciones exploradas, con la solucion como clave 
            self.explored_solutions[tuple_solution] = fitness
            self.total_evaluated_items += 1
            self.real_evaluated_items += 1

        # Devolvemos el fitness de la solucion
        return fitness


    
    def search(self, departure, goal):
        # Asignamos a la variable departure la ciudad de origen, es decir, en la que nos encontramos
        self.problema.departure = departure
        # Asignamos a la variable goal la ciudad de destino, es decir, en la que queremos llegar
        self.problema.goal = goal

        # Llamamos al metodo search en concreto el metodo A* para resolver el problema
        # En el archivo utils he cambiado el return de do_search llamando a la funcion que me calcula el coste
        # del camino que recorre el algoritmo, es decir, la distancia entre la ciudad de origen y la ciudad de destino
        return ut.AStar(self.problema).do_search()




        
    

PRUEBAS COMBINE (GENETIC)

In [60]:
# Probamos combine de genetic
# Evaluamos los nuevos individuos
# Tenemos 2 individuos nuevos (childs) y 5 individuos de la poblacion actual (gn.population)
cvrp = CVRP()
for individual in gn.population:
    print(individual)
    individual.evaluate(cvrp)
    print("Fitness: ", individual.fitness)


 Individual: [1, 0, 4, 5, 3, 2]
Fitness:  3244.0
 Individual: [3, 5, 4, 0, 2, 1]
Fitness:  3594.0
 Individual: [0, 3, 4, 2, 5, 1]
Fitness:  7220.0
 Individual: [2, 1, 0, 4, 3, 5]
Fitness:  13788.0
 Individual: [0, 4, 5, 1, 3, 2]
Fitness:  14003.0


In [ ]:
# TODO: Test here the methods to combine populations
# Probamos los metodos para combinar poblaciones
# Para este metodo combinaremos la poblacion actual population con los hijos (childs) que hemos obtenido del metodo crossover y mutation

# Llamamos al metodo evaluate de Individual para que asigne un fitness a cada individuo de la poblacion
for individual in gn.population:
    individual.evaluate_first_number()

for individual in childs:
    individual.evaluate_first_number()
    

# Probamos el metodo para combinar poblaciones
new_population = gn.combine4(childs)

# Imprimimos la nueva poblacion
print('Nueva población')
for individual in new_population:
    print(individual.genes)

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [119]:
# TODO: Test here the method to initialize the capacited vehicle routing problem
# # Inicializamos el vehiculo
# geneticAlgorithm = Genetic(50, 100, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1)

cvrp = CVRP()

# Comprobamos todas las variables del problema
print(cvrp.vehicles[0])
print(cvrp.warehouse)
for i in range(len(cvrp.parcels)):
    print(cvrp.parcels[i])

print(cvrp.parcels_dict)

# Obtenemos la longitud del diccionario



suma = cvrp.search(1, 2)
print(suma)






{'id': 0, 'capacity': 20}
11
{'id': 0, 'city': 9, 'weight': 5}
{'id': 1, 'city': 5, 'weight': 7}
{'id': 2, 'city': 7, 'weight': 11}
{'id': 3, 'city': 3, 'weight': 6}
{'id': 4, 'city': 3, 'weight': 6}
{'id': 5, 'city': 8, 'weight': 14}
{0: (9, 5), 1: (5, 7), 2: (7, 11), 3: (3, 6), 4: (3, 6), 5: (8, 14)}
899.0


In [120]:
# TODO: Test here the method to solve the search problem
# Resolvemos el problema de busqueda
# print(cvrp.search(11, 9))
# Hacemos la busqueda 

In [127]:
# TODO: Test here the method to solve the capacited vehicle routing problem
# Resolvemos el problema del CVRP
problem = "more_cases-low_dimension\problems\example.json"

geneticAlgorithm = Genetic(150, 1000, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1, False)

cvrp = CVRP(problem, algorithm = geneticAlgorithm)

cvrp()



explore:  {(1, 4, 0, 5, 3, 2): 3446.0, (0, 3, 4, 2, 5, 1): 4479.0, (3, 5, 4, 0, 2, 1): 4944.0, (3, 4, 1, 2, 0, 5): 7506.0, (2, 1, 0, 4, 3, 5): 8530.0, (4, 5, 0, 3, 1, 2): 9659.0, (3, 4, 0, 5, 2, 1): 10012.0, (0, 4, 5, 1, 3, 2): 10089.0, (1, 4, 0, 3, 5, 2): 10521.0, (2, 1, 4, 3, 5, 0): 10740.0, (5, 2, 3, 4, 0, 1): 10966.0, (0, 4, 2, 5, 3, 1): 11205.0, (4, 3, 1, 2, 5, 0): 11796.0, (1, 3, 2, 5, 4, 0): 11796.0, (0, 3, 1, 2, 5, 4): 11796.0, (0, 3, 4, 2, 1, 5): 11957.0, (5, 1, 3, 4, 0, 2): 12156.0, (4, 5, 3, 2, 1, 0): 12417.0, (4, 0, 2, 1, 3, 5): 12560.0, (2, 4, 3, 5, 1, 0): 12701.0, (1, 3, 2, 5, 0, 4): 12735.0, (0, 3, 2, 4, 5, 1): 13148.0, (0, 2, 3, 4, 5, 1): 13272.0, (4, 1, 5, 0, 3, 2): 13403.0, (3, 1, 2, 5, 0, 4): 13557.0, (2, 4, 1, 3, 5, 0): 13739.0, (0, 2, 1, 4, 5, 3): 13863.0, (4, 2, 0, 1, 3, 5): 13894.0, (4, 5, 3, 0, 2, 1): 14017.0, (3, 4, 0, 5, 1, 2): 14105.0, (0, 3, 1, 5, 4, 2): 14258.0, (5, 4, 1, 3, 0, 2): 14265.0, (2, 1, 5, 4, 0, 3): 14407.0, (3, 4, 1, 5, 0, 2): 14669.0, (3, 1, 5,

 Individual: [3, 0, 5, 1, 4, 2]

In [129]:
# Evaluar un individuo concreto
print(cvrp.evaluate([1, 4, 0, 5, 3, 2]))

3446.0


In [125]:
problem = "more_cases-low_dimension\problems\example.json"

geneticAlgorithm = Genetic(150, 1000, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, 1, False)

cvrp = CVRP(problem, algorithm = geneticAlgorithm)

cvrp.evaluate([1, 3, 0, 4, 2, 5])

3314.0

### 4.3. Estudio y mejora de los algoritmos

Una vez que los algoritmos han sido implementados, se debe **estudiar** su **rendimiento**. Para ello, se debe comparar la **calidad** de las **soluciones obtenidas**, así como las **diferentes estadísticas** que se consideren adecuadas (número de generaciones, tiempo de ejecución, etc.). Factores como el tamaño máximo del problema que se soporta sin un error de memoria, así como el efecto temporal de usar escenarios más complejos son otros factores a tener en cuenta. Además, se **pueden proponer** y se valorarán la incorporación de **técnicas** que **permitan acelerar** la **ejecución** de los **algoritmos**.

---

In [97]:
# TODO: Experiment here with the small problem
# Creamos un diccionario top 10 para guardar los mejores fitness(min) de cada ejecucion

top10 = []
problem = "more_cases-low_dimension\problems\example.json"

def run_ga(semilla):
    geneticAlgorithm = Genetic(250, 1000, "torneo", 'crossover', 0.8, 'mutation', 0.1, True, semilla, False)
    cvrp = CVRP(problem, algorithm = geneticAlgorithm)
    solution = cvrp()
    if len(top10) < 10:
        top10.append(solution)
        top10.sort(key=lambda x: x.fitness)
    else:
        for top10_fitness in top10:
            # Comprobamos el fitness de la solucion y lo guardamos en el diccionario si esta entre los 10 mejores
            if solution.fitness < top10_fitness:
                if len(top10) < 10:
                    top10.append(solution)
                    # Ordenamos la lista de menor a mayor
                    top10.sort(key=lambda x: x.fitness)
                    # Salimos del bucle for
                    break
                else:
                    top10.pop()
                    top10.append(solution)
                    # Salimos del bucle for
                    break

    return solution

# Crea una lista de hilos
threads = []

# Recorre la semilla y crea un hilo para cada elemento
for i in range(20):
    t = threading.Thread(target=run_ga, args=(i,))
    threads.append(t)

# Inicia todos los hilos
for t in threads:
    t.start()
    t.join()

# Espera a que todos los hilos terminen
for t in threads:
    t.join()


print("Top 10: ", top10)


explore:  {(3, 5, 0, 1, 2, 4): 3752.0, (2, 3, 5, 4, 0, 1): 4755.0, (2, 0, 4, 5, 1, 3): 5570.0, (4, 3, 5, 2, 1, 0): 7676.0, (4, 3, 2, 0, 1, 5): 7785.0, (2, 1, 0, 5, 4, 3): 7891.0, (2, 1, 3, 4, 0, 5): 8052.0, (3, 4, 0, 2, 1, 5): 8052.0, (2, 1, 4, 3, 0, 5): 8052.0, (4, 5, 2, 3, 0, 1): 8621.0, (3, 5, 2, 4, 0, 1): 8621.0, (3, 0, 1, 2, 4, 5): 8705.0, (5, 4, 3, 2, 1, 0): 9201.0, (2, 4, 5, 3, 0, 1): 9443.0, (5, 4, 2, 3, 0, 1): 9443.0, (5, 3, 2, 4, 0, 1): 9443.0, (3, 0, 2, 1, 5, 4): 9640.0, (2, 0, 4, 3, 5, 1): 10446.0, (4, 0, 5, 2, 1, 3): 10873.0, (3, 0, 5, 2, 1, 4): 10873.0, (0, 1, 5, 3, 4, 2): 10961.0, (5, 4, 3, 0, 1, 2): 11115.0, (1, 5, 3, 4, 2, 0): 11924.0, (1, 5, 4, 3, 2, 0): 11924.0, (4, 5, 2, 1, 3, 0): 12111.0, (5, 2, 3, 4, 0, 1): 12188.0, (4, 0, 2, 1, 3, 5): 12361.0, (3, 0, 2, 1, 4, 5): 12361.0, (0, 3, 4, 2, 1, 5): 12844.0, (0, 4, 3, 2, 1, 5): 12844.0, (5, 4, 2, 0, 1, 3): 12860.0, (0, 1, 4, 2, 3, 5): 12860.0, (5, 4, 2, 1, 3, 0): 12933.0, (5, 3, 2, 1, 4, 0): 12933.0, (3, 0, 4, 2, 1, 5): 

Exception in thread Thread-56 (run_ga):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\carlo\AppData\Local\Temp\ipykernel_14312\23080391.py", line 17, in run_ga
TypeError: '<' not supported between instances of 'float' and 'Individual'


explore:  {(4, 0, 3, 1, 5, 2): 4013.0, (3, 2, 4, 1, 0, 5): 4468.0, (4, 5, 3, 0, 2, 1): 10198.0, (3, 5, 2, 4, 0, 1): 10211.0, (4, 5, 0, 3, 2, 1): 10352.0, (3, 5, 4, 1, 2, 0): 11799.0, (3, 4, 5, 1, 2, 0): 11908.0, (1, 2, 4, 3, 5, 0): 12024.0, (0, 2, 4, 3, 1, 5): 12091.0, (1, 2, 3, 5, 4, 0): 12277.0, (4, 2, 0, 3, 1, 5): 12399.0, (0, 2, 3, 5, 4, 1): 12457.0, (4, 3, 2, 0, 5, 1): 12818.0, (3, 2, 4, 5, 1, 0): 12919.0, (0, 3, 4, 1, 2, 5): 13351.0, (1, 2, 0, 3, 4, 5): 13351.0, (1, 2, 4, 0, 3, 5): 13505.0, (3, 0, 4, 1, 2, 5): 13505.0, (3, 2, 1, 0, 4, 5): 14246.0, (0, 4, 1, 3, 5, 2): 15518.0, (0, 5, 4, 3, 2, 1): 15541.0, (3, 2, 4, 0, 5, 1): 15695.0, (3, 4, 0, 1, 2, 5): 15764.0, (5, 3, 4, 0, 2, 1): 15874.0, (0, 4, 5, 3, 2, 1): 16028.0, (5, 3, 0, 4, 2, 1): 16028.0, (4, 3, 0, 2, 5, 1): 16138.0, (1, 2, 5, 4, 3, 0): 16530.0, (3, 1, 5, 4, 2, 0): 16748.0, (0, 3, 1, 2, 4, 5): 17129.0, (3, 5, 1, 0, 4, 2): 17365.0, (5, 3, 4, 1, 2, 0): 17475.0, (1, 2, 3, 0, 5, 4): 17869.0, (0, 3, 4, 5, 1, 2): 18202.0, (3, 5

Exception in thread Thread-57 (run_ga):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\carlo\AppData\Local\Temp\ipykernel_14312\23080391.py", line 17, in run_ga
TypeError: '<' not supported between instances of 'float' and 'Individual'


explore:  {(3, 4, 5, 1, 0, 2): 3592.0, (4, 3, 1, 0, 5, 2): 7546.0, (1, 0, 4, 2, 5, 3): 9762.0, (2, 3, 1, 5, 0, 4): 11272.0, (0, 5, 1, 4, 2, 3): 12377.0, (1, 0, 3, 5, 2, 4): 14002.0, (4, 3, 1, 5, 0, 2): 15992.0, (5, 4, 1, 2, 0, 3): 17219.0, (2, 3, 4, 1, 0, 5): 17292.0, (5, 2, 3, 4, 0, 1): 17753.0, (2, 4, 1, 0, 3, 5): 17782.0, (0, 5, 3, 4, 1, 2): 18127.0, (0, 4, 1, 2, 3, 5): 18246.0, (2, 4, 5, 1, 0, 3): 18548.0, (0, 5, 2, 3, 4, 1): 19296.0, (0, 5, 2, 4, 3, 1): 19296.0, (1, 5, 2, 3, 4, 0): 20110.0, (3, 4, 0, 2, 1, 5): 20385.0, (2, 1, 4, 3, 0, 5): 20385.0, (4, 3, 0, 2, 1, 5): 20385.0, (5, 2, 4, 1, 0, 3): 20424.0, (1, 5, 0, 3, 4, 2): 20478.0, (5, 1, 0, 4, 2, 3): 20914.0, (4, 1, 5, 0, 3, 2): 20978.0, (3, 1, 5, 0, 4, 2): 20978.0, (5, 0, 2, 3, 4, 1): 21157.0, (3, 0, 5, 2, 4, 1): 21279.0, (4, 0, 3, 2, 1, 5): 21375.0, (4, 0, 2, 1, 5, 3): 21730.0, (1, 5, 3, 4, 0, 2): 21981.0, (2, 4, 5, 0, 3, 1): 22147.0, (3, 4, 0, 2, 5, 1): 22175.0, (4, 3, 0, 2, 5, 1): 22175.0, (3, 0, 4, 5, 1, 2): 22654.0, (0, 4,

...

explore:  {(4, 3, 2, 1, 5, 0): 3768.0, (1, 4, 0, 3, 2, 5): 5808.0, (5, 4, 2, 0, 3, 1): 6588.0, (3, 5, 2, 1, 4, 0): 8886.0, (4, 5, 2, 1, 3, 0): 8886.0, (4, 0, 2, 5, 1, 3): 9602.0, (3, 5, 2, 4, 1, 0): 9818.0, (2, 3, 0, 5, 1, 4): 9889.0, (1, 4, 0, 3, 5, 2): 10173.0, (3, 0, 2, 4, 5, 1): 11644.0, (4, 0, 2, 3, 5, 1): 11644.0, (5, 1, 3, 4, 0, 2): 11845.0, (2, 4, 3, 5, 0, 1): 13045.0, (4, 2, 5, 1, 3, 0): 13069.0, (2, 1, 4, 5, 0, 3): 13129.0, (5, 0, 2, 1, 3, 4): 13246.0, (2, 0, 3, 4, 5, 1): 13456.0, (1, 4, 2, 3, 0, 5): 13489.0, (2, 1, 5, 3, 4, 0): 13785.0, (3, 4, 0, 2, 5, 1): 13857.0, (2, 1, 4, 3, 5, 0): 14064.0, (4, 0, 3, 5, 2, 1): 14077.0, (5, 4, 1, 3, 0, 2): 14339.0, (1, 3, 5, 4, 0, 2): 14339.0, (4, 2, 3, 5, 0, 1): 14351.0, (2, 0, 4, 5, 1, 3): 14395.0, (5, 1, 4, 3, 2, 0): 14780.0, (2, 3, 1, 4, 5, 0): 14919.0, (5, 2, 1, 4, 3, 0): 15003.0, (1, 5, 4, 3, 0, 2): 15072.0, (3, 4, 2, 5, 0, 1): 15258.0, (2, 3, 4, 0, 5, 1): 15267.0, (2, 0, 1, 3, 4, 5): 15301.0, (2, 1, 4, 0, 5, 3): 15351.0, (4, 0, 2, 1

Exception in thread Thread-64 (run_ga):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\carlo\AppData\Local\Temp\ipykernel_14312\23080391.py", line 17, in run_ga
TypeError: '<' not supported between instances of 'float' and 'Individual'


explore:  {(1, 0, 3, 5, 4, 2): 3244.0, (3, 5, 1, 4, 2, 0): 6941.0, (4, 5, 2, 1, 0, 3): 7032.0, (2, 1, 5, 3, 0, 4): 8793.0, (1, 0, 4, 3, 5, 2): 8939.0, (1, 0, 4, 2, 3, 5): 11229.0, (4, 1, 5, 0, 2, 3): 12879.0, (5, 3, 1, 0, 4, 2): 13016.0, (5, 4, 2, 3, 1, 0): 13991.0, (5, 3, 2, 4, 1, 0): 13991.0, (5, 2, 4, 0, 1, 3): 14078.0, (1, 0, 4, 5, 3, 2): 14102.0, (4, 5, 2, 1, 3, 0): 14526.0, (3, 1, 5, 0, 4, 2): 14730.0, (3, 4, 5, 2, 0, 1): 14746.0, (0, 4, 1, 2, 5, 3): 15003.0, (3, 1, 4, 0, 5, 2): 15852.0, (3, 0, 4, 2, 1, 5): 16160.0, (5, 2, 1, 4, 3, 0): 16751.0, (3, 1, 2, 4, 5, 0): 17282.0, (4, 1, 2, 3, 5, 0): 17282.0, (5, 1, 0, 4, 3, 2): 17516.0, (4, 2, 5, 3, 1, 0): 17632.0, (4, 5, 1, 0, 2, 3): 18164.0, (0, 4, 2, 1, 5, 3): 18477.0, (0, 3, 2, 1, 5, 4): 18477.0, (0, 3, 2, 4, 5, 1): 18513.0, (2, 3, 0, 4, 5, 1): 18513.0, (0, 4, 2, 3, 5, 1): 18513.0, (0, 4, 2, 1, 3, 5): 18566.0, (5, 4, 0, 3, 1, 2): 18566.0, (5, 3, 0, 4, 1, 2): 18566.0, (3, 5, 1, 2, 4, 0): 18598.0, (4, 5, 1, 2, 3, 0): 18598.0, (5, 3, 2

Exception in thread Thread-65 (run_ga):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\carlo\AppData\Local\Temp\ipykernel_14312\23080391.py", line 17, in run_ga
TypeError: '<' not supported between instances of 'float' and 'Individual'


explore:  {(5, 0, 4, 2, 3, 1): 3760.0, (2, 4, 5, 3, 0, 1): 7525.0, (0, 2, 3, 1, 4, 5): 7544.0, (2, 5, 0, 4, 3, 1): 9859.0, (0, 3, 5, 1, 2, 4): 12003.0, (5, 0, 2, 3, 1, 4): 14163.0, (3, 2, 1, 4, 5, 0): 15244.0, (4, 3, 5, 0, 2, 1): 15490.0, (3, 1, 4, 5, 0, 2): 15499.0, (0, 2, 4, 1, 3, 5): 15749.0, (5, 0, 4, 3, 2, 1): 15936.0, (1, 4, 5, 0, 2, 3): 16552.0, (1, 3, 5, 0, 2, 4): 16552.0, (5, 0, 4, 1, 3, 2): 16767.0, (1, 3, 4, 0, 2, 5): 16768.0, (0, 2, 1, 4, 3, 5): 16768.0, (1, 4, 3, 0, 2, 5): 16768.0, (3, 0, 4, 2, 1, 5): 17463.0, (0, 2, 5, 4, 1, 3): 17497.0, (3, 2, 4, 1, 0, 5): 17812.0, (0, 2, 5, 1, 3, 4): 18293.0, (2, 5, 0, 1, 4, 3): 19803.0, (1, 0, 3, 5, 4, 2): 19869.0, (2, 5, 3, 1, 4, 0): 19907.0, (4, 2, 1, 0, 3, 5): 19973.0, (2, 4, 1, 0, 3, 5): 20099.0, (2, 5, 3, 1, 0, 4): 20382.0, (2, 5, 1, 4, 3, 0): 20703.0, (5, 4, 2, 3, 1, 0): 20828.0, (2, 3, 5, 4, 1, 0): 20828.0, (0, 1, 3, 4, 2, 5): 21139.0, (0, 1, 4, 3, 2, 5): 21139.0, (2, 3, 0, 1, 4, 5): 21265.0, (5, 0, 4, 1, 2, 3): 21358.0, (1, 2, 

In [85]:
# Top 10 
print("Top 10: ")
for i in range(len(top10)):
    print("Top ", i+1, ": ", top10[i], "Fitness: " ,top10[i].fitness)


Top 10: 
Top  1 :   Individual: [1, 2, 3, 0, 5, 4] Fitness:  2659.0
Top  2 :   Individual: [0, 1, 2, 4, 5, 3] Fitness:  2659.0
Top  3 :   Individual: [3, 4, 5, 1, 0, 2] Fitness:  3047.0
Top  4 :   Individual: [1, 4, 5, 0, 2, 3] Fitness:  3225.0
Top  5 :   Individual: [5, 4, 2, 1, 3, 0] Fitness:  3788.0
Top  6 :   Individual: [2, 1, 3, 5, 4, 0] Fitness:  3857.0
Top  7 :   Individual: [1, 3, 2, 4, 5, 0] Fitness:  3865.0
Top  8 :   Individual: [1, 4, 3, 2, 0, 5] Fitness:  3883.0
Top  9 :   Individual: [2, 3, 1, 4, 0, 5] Fitness:  3948.0
Top  10 :   Individual: [0, 2, 3, 1, 4, 5] Fitness:  5736.0


In [83]:
# Evaluamos el fitness 
print("Fitness: ", cvrp.evaluate([1, 2, 3, 0, 5, 4]))

Fitness:  30566.0


In [ ]:
# TODO: Experiment here with the medium problem

In [ ]:
# TODO: Experiment here with the large problem

### 5. Entrega y evaluación

Al igual que la práctica anterior, esta se debe **hacer en pares**. No obstante, en **casos excepcionales** se permite realizarla **individualmente**. **La fecha límite para subir la práctica es el 18 de diciembre de 2022 a las 23:55**. Las **entrevistas y evaluaciones** se realizarán la **semana siguiente**.

Algunas consideraciones:

* **En caso de que no se haya entregado la primera práctica, o se haya sacado menos de un cuatro, se podrán entregar conjuntamente en esta fecha. No obstante, se considerará únicamente un 90% de la nota global de prácticas**.
* Está práctica supone el **70%** de la **nota** en este apartado.
* La práctica se **evaluará** mediante una **entrevista individual** con el profesorado. Las fechas de las entrevistas se publicarán con antelación.
*  Se proporcionará un **conjunto** de **casos** de **prueba preliminares** (varios mapas e instancias) que se **deben resolver correctamente**. En caso contrario, la práctica se considerará suspensa.
* La **entrevista** consistirá en una serie de **preguntas** acerca del **código**.

**Por último, para la evaluación no continua se requirirá la implementación del algoritmo de búsqueda por ascenso de colinas. Además, este se deberá utilizar para inicializar la población del algoritmo genético, en lugar de que sea aleatoria.**